data_pipeline.ipynb


# Load data
## Videos data

In [ ]:
dbname='ds-wizards'
user='wizard'
host='192.95.32.117'
password='GaG23jVxZhMnQaU53r8o'

VQUERY = "select post_id, url from videos where status='ok'"

vres = fetch(dbname, user, host, password, VQUERY)
vres = [(post_id.split("_")[1], url) for post_id, url in vres]

## Tags

In [ ]:
dbname='ds-content-tags'
user='ds-content-tags'
password='0fXjWl592vNf1gYvIw8w'
host='192.95.32.117'

TQUERY = "select id, tags from videos where tags is not NULL"
TAGS = "select tag_id, name, path from content_tags"

tres = fetch(dbname, user, host, password, TQUERY)

tags = { 
    tag_id: (name, path) for (tag_id, name, path) in fetch(
        dbname, user, host, password, TAGS) 
}

## Join videos with tags

In [ ]:
videos = inner_join(tres, vres)
filtered, t2i, i2t = filter_videos(videos, 10)
print("Found %d videos with %d unique tags" % (len(filtered), len(t2i)))

# TEST

In [ ]:
import time

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
work = filtered[:LIMIT]

import time

def run_and_measure(fun):
    start_time = time.time()
    fun()
    end_time = time.time()
    elapsed = end_time - start_time
    print("Elapsed time was %g seconds [%g]" % (elapsed, elapsed/len(work)))

fsq = lambda : fetch_sq(work, model_path=MODEL_PATH, data_path=os.path.join(DATA_PATH, 'seq'))
fmp = lambda : fetch_mp(work, nprod=4, model_path=MODEL_PATH, data_path=os.path.join(DATA_PATH, 'mp'))

run_and_measure(fmp)